In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('embedding_data_df.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3154 entries, 0 to 3153
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              3154 non-null   int64 
 1   doc_id                  3154 non-null   int64 
 2   text-embedding-3-large  3009 non-null   object
 3   text-embedding-3-small  2996 non-null   object
dtypes: int64(2), object(2)
memory usage: 98.7+ KB


In [3]:
df.head()

,Unnamed: 0,doc_id,text-embedding-3-large,text-embedding-3-small
0,0,3721,"[-0.003465590998530388, 0.0029300767928361893,...","[0.0011742237256839871, 0.030985577031970024, ..."
1,1,2833,"[0.019412847235798836, -0.012032884173095226, ...","[0.025459323078393936, -0.021006928756833076, ..."
2,2,3371,"[0.007026161532849073, 0.004458976443856955, -...","[0.01621224172413349, -0.004911521449685097, 0..."
3,3,683,"[0.0002814432082232088, -0.007989478297531605,...","[0.04488416016101837, -0.027948830276727676, 0..."
4,4,6173,"[-8.119385893223807e-05, -0.027250811457633972...","[-0.007460186257958412, -0.0025699581019580364..."


In [4]:
entry_type = type(df['text-embedding-3-large'].iloc[0])
print(entry_type)
print(df['text-embedding-3-large'].iloc[0])

<class 'str'>
[-0.003465590998530388, 0.0029300767928361893, -0.016648394986987114, -0.010683171451091766, -0.06377365440130234, 0.007348069921135902, 0.02086472138762474, -0.01776009425520897, -0.010778073221445084, 0.0012743885163217783, 0.04964692145586014, -0.00581270270049572, -0.014940171502530575, 0.012811671942472458, 0.053334515541791916, 0.0030148099176585674, -0.042841143906116486, -0.0353032723069191, 0.0048738582991063595, -0.0015074049588292837, 0.020308872684836388, -0.007124374154955149, 0.009930740110576153, -0.0036774242762476206, 0.012303272262215614, 0.006253316067159176, 0.042732685804367065, -0.016214560717344284, -0.03253757953643799, 0.031805481761693954, 0.0521685853600502, -0.03074801154434681, 0.021108753979206085, -0.006765104830265045, -0.0026555408257991076, -0.045715298503637314, 0.008846154436469078, 0.01349631603807211, 0.029961686581373215, -0.009639257565140724, -0.009510463103652, -0.0057347482070326805, -0.01258119661360979, 0.0012328692246228456, -

In [3]:
import ast


# Fill NaN values with an empty list
df['text-embedding-3-large'] = df['text-embedding-3-large'].apply(lambda x: x if pd.notna(x) else '[]')
df['text-embedding-3-small'] = df['text-embedding-3-small'].apply(lambda x: x if pd.notna(x) else '[]')

# Convert the 'text-embedding-3-large' and 'text-embedding-3-small' columns to numpy arrays of floats
df['text-embedding-3-large'] = df['text-embedding-3-large'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))
df['text-embedding-3-small'] = df['text-embedding-3-small'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))

# Store the entire dataframe in a new variable called new_data
df

,Unnamed: 0,doc_id,text-embedding-3-large,text-embedding-3-small
0,0,3721,"[-0.003465590998530388, 0.0029300767928361893,...","[0.0011742237256839871, 0.030985577031970024, ..."
1,1,2833,"[0.019412847235798836, -0.012032884173095226, ...","[0.025459323078393936, -0.021006928756833076, ..."
2,2,3371,"[0.007026161532849073, 0.004458976443856955, -...","[0.01621224172413349, -0.004911521449685097, 0..."
3,3,683,"[0.0002814432082232088, -0.007989478297531605,...","[0.04488416016101837, -0.027948830276727676, 0..."
4,4,6173,"[-8.119385893223807e-05, -0.027250811457633972...","[-0.007460186257958412, -0.0025699581019580364..."
...,...,...,...,...
3149,3149,2324,"[-0.029096072539687157, 0.01301939133554697, -...","[0.015040543861687183, 0.0009863896993920207, ..."
3150,3150,2631,"[0.0010496911127120256, -0.01915811561048031, ...","[-0.022074943408370018, 0.013342805206775665, ..."
3151,3151,3020,"[-0.027409426867961884, -0.0003570443368516862...","[0.049575261771678925, -0.017476992681622505, ..."
3152,3152,3470,"[0.016199158504605293, -0.01308600977063179, -...","[-0.001969351898878813, -0.011167383752763271,..."


In [6]:
entry_type = type(new_data['text-embedding-3-large'].iloc[0])
print(entry_type)
print(new_data['text-embedding-3-large'].iloc[0])

<class 'numpy.ndarray'>
[-0.00346559  0.00293008 -0.01664839 ...  0.01415385  0.00740908
 -0.00652446]


`!pip install pymilvus dspy openai nltk`

In [7]:
new_data.head()

,Unnamed: 0,doc_id,text-embedding-3-large,text-embedding-3-small
0,0,3721,"[-0.003465590998530388, 0.0029300767928361893,...","[0.0011742237256839871, 0.030985577031970024, ..."
1,1,2833,"[0.019412847235798836, -0.012032884173095226, ...","[0.025459323078393936, -0.021006928756833076, ..."
2,2,3371,"[0.007026161532849073, 0.004458976443856955, -...","[0.01621224172413349, -0.004911521449685097, 0..."
3,3,683,"[0.0002814432082232088, -0.007989478297531605,...","[0.04488416016101837, -0.027948830276727676, 0..."
4,4,6173,"[-8.119385893223807e-05, -0.027250811457633972...","[-0.007460186257958412, -0.0025699581019580364..."


In [12]:
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType, utility

# ollama = Client(host='http://ollama:11434')

# def get_ollama_embeddings(text):
#     embedding_output = ollama.embeddings(
#         model='mxbai-embed-large',
#         prompt=text
#     )
#     return np.array(embedding_output.embedding, dtype=np.float32)  # Convert to float32

# Connect to Milvus server - remove http:// prefix
connections.connect(host='milvus', port='19530')

# Generate embeddings using Ollama


# Define collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="doc_id", dtype=DataType.INT64),
    # FieldSchema(name="text_embedding_large", dtype=DataType.FLOAT_VECTOR, dim=len(df['text-embedding-3-large'].iloc[0])),
    FieldSchema(name="text_embedding_small", dtype=DataType.FLOAT_VECTOR, dim=len(new_data['text-embedding-3-small'].iloc[0]))
]

schema = CollectionSchema(fields, "Text embeddings collection")
collection_name = "text_embeddings"

# Drop existing collection if it exists
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

# Create collection
collection = Collection(collection_name, schema)

# Filter out rows with missing or NaN values in any of the embedding columns
filtered_data = new_data.dropna(subset=['text-embedding-3-large', 'text-embedding-3-small'])

# Ensure the number of rows in all fields is consistent
filtered_data = filtered_data.dropna(subset=['text-embedding-3-large', 'text-embedding-3-small'])

# Truncate all fields to 3009 rows
# truncated_length = 3009
# filtered_data = filtered_data.iloc[:truncated_length]

# Insert data into the collection
collection.insert([
    filtered_data.index.tolist(),  # id
    filtered_data['doc_id'].tolist(),  # doc_id
    # filtered_data['text-embedding-3-large'].tolist(),  # text_embedding_large
    filtered_data['text-embedding-3-small'].tolist()  # text_embedding_small
])


# Create index for vector field
index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 56}
}
collection.create_index(field_name="text_embedding_small",index_params=index_params, index_name="text_embedding_small_index")
collection.load()

print(f"Created collection with {collection.num_entities} entities")
connections.disconnect("default")

RPC error: [create_index], <MilvusException: (code=1, message=cannot create index on non-existed field: 'text_embedding_small)>, <Time:{'RPC start': '2025-01-26 21:14:47.472219', 'RPC error': '2025-01-26 21:14:47.474191'}>


MilvusException: <MilvusException: (code=1, message=cannot create index on non-existed field: 'text_embedding_small)>

In [14]:
# Create index for vector field
index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 56}
}
collection.create_index(field_name="text_embedding_small",index_params=index_params, index_name="text_embedding_small_index")
collection.load()

print(f"Created collection with {collection.num_entities} entities")
connections.disconnect("default")

Created collection with 2967 entities


In [9]:
# Ensure there are no hidden NaN values
filtered_data = filtered_data.dropna(subset=['text-embedding-3-large', 'text-embedding-3-small'])

# Check the lengths of the lists
print(len(filtered_data.index.tolist()))
print(len(filtered_data['doc_id'].tolist()))
print(len(filtered_data['text-embedding-3-large'].tolist()))
print(len(filtered_data['text-embedding-3-small'].tolist()))

3009
3009
3009
3009


In [10]:
# Truncate all fields to 3009 rows
# truncated_length = 3009
# filtered_data = filtered_data.iloc[:truncated_length]

# Insert data into the collection
collection.insert([
    filtered_data.index.tolist(),  # id
    filtered_data['doc_id'].tolist(),  # doc_id
    filtered_data['text-embedding-3-large'].tolist(),  # text_embedding_large
    filtered_data['text-embedding-3-small'].tolist()  # text_embedding_small
])

: 

In [1]:
import pandas as pd
import numpy as np
import ast

# Load the CSV file
df = pd.read_csv('embedding_data_df.csv')

# Check for NaN values
print("NaN values in each column:")
print(df.isna().sum())

# Check the lengths of the embeddings
embedding_large_lengths = df['text-embedding-3-large'].dropna().apply(lambda x: len(ast.literal_eval(x)))
embedding_small_lengths = df['text-embedding-3-small'].dropna().apply(lambda x: len(ast.literal_eval(x)))

print("\nUnique lengths of text-embedding-3-large:")
print(embedding_large_lengths.unique())

print("\nUnique lengths of text-embedding-3-small:")
print(embedding_small_lengths.unique())

# Ensure all embeddings are properly formatted
def is_valid_embedding(embedding):
    try:
        emb_list = ast.literal_eval(embedding)
        return isinstance(emb_list, list) and all(isinstance(i, (int, float)) for i in emb_list)
    except:
        return False

invalid_large_embeddings = df['text-embedding-3-large'].dropna().apply(lambda x: not is_valid_embedding(x))
invalid_small_embeddings = df['text-embedding-3-small'].dropna().apply(lambda x: not is_valid_embedding(x))

print("\nInvalid text-embedding-3-large entries:")
print(df[invalid_large_embeddings])

print("\nInvalid text-embedding-3-small entries:")
print(df[invalid_small_embeddings])

NaN values in each column:
Unnamed: 0                  0
doc_id                      0
text-embedding-3-large    145
text-embedding-3-small    158
dtype: int64

Unique lengths of text-embedding-3-large:
[3072]

Unique lengths of text-embedding-3-small:
[1536]

Invalid text-embedding-3-large entries:


/tmp/ipykernel_84563/1892522405.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df[invalid_large_embeddings])


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [6]:
import pandas as pd
import numpy as np
import ast

# Load the CSV file
df = pd.read_csv('embedding_data_df.csv')

# Check for NaN values
print("NaN values in each column:")
print(df.isna().sum())

# Check the lengths of the embeddings
embedding_large_lengths = df['text-embedding-3-large'].dropna().apply(lambda x: len(ast.literal_eval(x)))
embedding_small_lengths = df['text-embedding-3-small'].dropna().apply(lambda x: len(ast.literal_eval(x)))

print("\nUnique lengths of text-embedding-3-large:")
print(embedding_large_lengths.unique())

print("\nUnique lengths of text-embedding-3-small:")
print(embedding_small_lengths.unique())

# Ensure all embeddings are properly formatted
def is_valid_embedding(embedding):
    try:
        emb_list = ast.literal_eval(embedding)
        return isinstance(emb_list, list) and all(isinstance(i, (int, float)) for i in emb_list)
    except:
        return False

# Create boolean series for invalid embeddings
invalid_large_embeddings = df['text-embedding-3-large'].dropna().apply(lambda x: not is_valid_embedding(x))
invalid_small_embeddings = df['text-embedding-3-small'].dropna().apply(lambda x: not is_valid_embedding(x))

# Align the boolean series with the DataFrame index
invalid_large_embeddings = invalid_large_embeddings.reindex(df.index, fill_value=False)
invalid_small_embeddings = invalid_small_embeddings.reindex(df.index, fill_value=False)

print("\nInvalid text-embedding-3-large entries:")
print(df[invalid_large_embeddings])

print("\nInvalid text-embedding-3-small entries:")
print(df[invalid_small_embeddings])

# Filter out rows with invalid embeddings
df = df[~invalid_large_embeddings & ~invalid_small_embeddings]

# Filter out rows with missing or NaN values in any of the embedding columns
df = df.dropna(subset=['text-embedding-3-large', 'text-embedding-3-small'])

# Ensure the number of rows in all fields is consistent by truncating to the minimum length
min_length = min(len(df.index), 
                 len(df['doc_id']), 
                 len(df['text-embedding-3-large']), 
                 len(df['text-embedding-3-small']))

df = df.iloc[:min_length]

# Convert the 'text-embedding-3-large' and 'text-embedding-3-small' columns to numpy arrays of floats
df['text-embedding-3-large'] = df['text-embedding-3-large'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))
df['text-embedding-3-small'] = df['text-embedding-3-small'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))

# Store the cleaned dataframe in a new variable called new_data
new_data = df

NaN values in each column:
Unnamed: 0                  0
doc_id                      0
text-embedding-3-large    145
text-embedding-3-small    158
dtype: int64

Unique lengths of text-embedding-3-large:
[3072]

Unique lengths of text-embedding-3-small:
[1536]

Invalid text-embedding-3-large entries:
Empty DataFrame
Columns: [Unnamed: 0, doc_id, text-embedding-3-large, text-embedding-3-small]
Index: []

Invalid text-embedding-3-small entries:
Empty DataFrame
Columns: [Unnamed: 0, doc_id, text-embedding-3-large, text-embedding-3-small]
Index: []


---

In [1]:
from pymilvus import connections, Collection

# Connect to Milvus server
connections.connect(host='milvus', port='19530')

In [2]:
# Load the collection
collection_name = "text_embeddings"
collection = Collection(collection_name)
collection.load()

In [14]:
# Define the primary key value you want to query
primary_key_value = 0

# Perform the query
results = collection.query(expr="id == 0",output_fields=["text_embedding_small"])
query = results[0]['text_embedding_small']
# Print the results
for result in results:
    print(result)
print(query)

{'text_embedding_small': [np.float32(0.0011742237), np.float32(0.030985577), np.float32(0.0062504136), np.float32(0.007261866), np.float32(0.012331242), np.float32(0.053007383), np.float32(-0.011004847), np.float32(0.01062328), np.float32(-0.02151305), np.float32(0.010126639), np.float32(0.011047242), np.float32(-0.00078281586), np.float32(-0.041911688), np.float32(0.016013656), np.float32(0.05533312), np.float32(0.015626034), np.float32(0.046563156), np.float32(-0.012179827), np.float32(-0.0395375), np.float32(0.032536067), np.float32(-0.013372977), np.float32(-0.009351394), np.float32(0.008969828), np.float32(-0.0048331684), np.float32(0.024832068), np.float32(-0.04646625), np.float32(-0.0024771502), np.float32(-0.005108744), np.float32(-0.04033697), np.float32(-0.012621959), np.float32(0.04905848), np.float32(-0.015747165), np.float32(-0.0009773842), np.float32(-0.0430261), np.float32(0.011065412), np.float32(0.035297878), np.float32(0.055187758), np.float32(-0.016522411), np.float3

In [15]:
# Define the search parameters
search_params = {
    "metric_type": "COSINE",
    "params": {"nprobe": 10}
}

# Define the query vector (example vector)
query_vector = query  # Replace with your actual query vector

# Perform the search
results = collection.search(
    data=[query_vector],
    anns_field="text_embedding_small",
    param=search_params,
    limit=10,
    expr=None
)

# Print the results
for hit in results[0]:
    print(f"ID: {hit.id}, Distance: {hit.distance}")

ID: 0, Distance: 1.000000238418579
ID: 2534, Distance: 0.7820424437522888
ID: 1112, Distance: 0.7538990378379822
ID: 1510, Distance: 0.7227424383163452
ID: 70, Distance: 0.7219550013542175
ID: 895, Distance: 0.717268705368042
ID: 932, Distance: 0.7154818177223206
ID: 600, Distance: 0.7141258716583252
ID: 2825, Distance: 0.7071082592010498
ID: 2784, Distance: 0.7009242177009583


In [ ]:
# Print the results
for result in results:
    print(f"ID: {result.id}, Distance: {result.distance}")

---

In [17]:
import pandas as pd
import numpy as np

In [18]:
json_df = pd.read_csv('json_data_df.csv')
json_df.head()
json_df.info()
json_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3154 entries, 0 to 3153
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      3154 non-null   int64  
 1   doc_id          3154 non-null   int64  
 2   index           3154 non-null   int64  
 3   link_path       2658 non-null   object 
 4   abstract        2656 non-null   object 
 5   all_authors     3154 non-null   object 
 6   pdf_path        2600 non-null   object 
 7   title           3154 non-null   object 
 8   publisher       3136 non-null   object 
 9   year_published  3154 non-null   int64  
 10  volume          2243 non-null   object 
 11  date_uploaded   2170 non-null   object 
 12  keywords        3154 non-null   object 
 13  start           986 non-null    float64
 14  end             985 non-null    float64
 15  paragraphs      3154 non-null   object 
 16  status          3154 non-null   int64  
 17  exception       3154 non-null   i

,Unnamed: 0,doc_id,index,year_published,start,end,status,exception
count,3154.000000,3154.000000,3154.000000,3154.000000,986.000000,985.000000,3154.000000,3154.000000
mean,1576.500000,2617.158529,2617.158529,2000.374445,175.803245,186.081218,1.891249,0.085606
std,910.625701,1523.436884,1523.436884,11.542785,133.080365,132.217983,0.311376,0.643809
min,0.000000,2.000000,2.000000,1922.000000,2.000000,3.000000,1.000000,0.000000
25%,788.250000,1311.250000,1311.250000,1992.000000,64.000000,75.000000,2.000000,0.000000
50%,1576.500000,2706.500000,2706.500000,1998.000000,148.500000,160.000000,2.000000,0.000000
75%,2364.750000,3747.500000,3747.500000,2009.000000,264.000000,272.000000,2.000000,0.000000
max,3153.000000,6185.000000,6185.000000,2023.000000,634.000000,646.000000,2.000000,5.000000


In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, db
import pandas as pd
import numpy as np
import ast

In [9]:

# Connect to Milvus server
connections.connect(host='milvus', port='19530')

# Load the JSON data into a DataFrame
json_data = pd.read_csv('json_data_df.csv')

# Load the embeddings data into a DataFrame
embedding_data = pd.read_csv('embedding_data_df.csv')

# Ensure there are no hidden NaN values in the embeddings
embedding_data = embedding_data.dropna(subset=['text-embedding-3-large', 'text-embedding-3-small'])

# Convert the embeddings from strings to lists of floats
embedding_data['text-embedding-3-large'] = embedding_data['text-embedding-3-large'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))
embedding_data['text-embedding-3-small'] = embedding_data['text-embedding-3-small'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))

# Merge the JSON data with the embeddings data on the 'doc_id' column
merged_data = pd.merge(json_data, embedding_data, on='doc_id')

# Define collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="doc_id", dtype=DataType.INT64),
    FieldSchema(name="index", dtype=DataType.INT64),
    FieldSchema(name="link_path", dtype=DataType.VARCHAR, max_length=2150),
    FieldSchema(name="abstract", dtype=DataType.VARCHAR, max_length=3209),
    FieldSchema(name="all_authors", dtype=DataType.VARCHAR, max_length=875),
    FieldSchema(name="pdf_path", dtype=DataType.VARCHAR, max_length=212),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=251),
    FieldSchema(name="publisher", dtype=DataType.VARCHAR, max_length=96),
    FieldSchema(name="year_published", dtype=DataType.INT64),
    FieldSchema(name="volume", dtype=DataType.VARCHAR, max_length=5),
    FieldSchema(name="date_uploaded", dtype=DataType.VARCHAR, max_length=10),
    FieldSchema(name="keywords", dtype=DataType.VARCHAR, max_length=285),
    FieldSchema(name="start", dtype=DataType.FLOAT),
    FieldSchema(name="end", dtype=DataType.FLOAT),
    # FieldSchema(name="paragraphs", dtype=DataType.VARCHAR, max_length=10000),
    FieldSchema(name="status", dtype=DataType.INT64),
    FieldSchema(name="exception", dtype=DataType.INT64),
    # FieldSchema(name="text_embedding_large", dtype=DataType.FLOAT_VECTOR, dim=3072),
    FieldSchema(name="text_embedding_small", dtype=DataType.FLOAT_VECTOR, dim=1536)
]

schema = CollectionSchema(fields, "Document data collection with embeddings")
collection_name = "doc_data"

# Drop existing collection if it exists
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

# Create collection
collection = Collection(collection_name, schema)

# Filter out rows with missing or NaN values in any of the required columns
filtered_data = merged_data.dropna(subset=['doc_id', 'index', 'title', 'year_published', 'keywords', 'paragraphs', 'status', 'exception', 'text-embedding-3-large', 'text-embedding-3-small'])

# Insert data into the collection
collection.insert([
    filtered_data.index.tolist(),  # id
    filtered_data['doc_id'].tolist(),  # doc_id
    filtered_data['index'].tolist(),  # index
    filtered_data['link_path'].fillna('').tolist(),  # link_path
    filtered_data['abstract'].fillna('').tolist(),  # abstract
    filtered_data['all_authors'].tolist(),  # all_authors
    filtered_data['pdf_path'].fillna('').tolist(),  # pdf_path
    filtered_data['title'].tolist(),  # title
    filtered_data['publisher'].fillna('').tolist(),  # publisher
    filtered_data['year_published'].tolist(),  # year_published
    filtered_data['volume'].fillna('').tolist(),  # volume
    filtered_data['date_uploaded'].fillna('').tolist(),  # date_uploaded
    filtered_data['keywords'].tolist(),  # keywords
    filtered_data['start'].fillna(0).tolist(),  # start
    filtered_data['end'].fillna(0).tolist(),  # end
    # filtered_data['paragraphs'].tolist(),  # paragraphs
    filtered_data['status'].tolist(),  # status
    filtered_data['exception'].tolist(),  # exception
    # filtered_data['text-embedding-3-large'].tolist(),  # text_embedding_large
    filtered_data['text-embedding-3-small'].tolist()  # text_embedding_small
])

index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 56}
}
collection.create_index(field_name="text_embedding_small",index_params=index_params, index_name="text_embedding_small_index")
# Load the collection
collection.load()

print(f"Created collection with {collection.num_entities} entities")
connections.disconnect("default")

Created collection with 2967 entities


In [7]:
max_lengths = {}
for column in json_data.columns:
    if json_data[column].dtype == 'object':
        max_lengths[column] = json_data[column].dropna().apply(len).max()
    else:
        max_lengths[column] = None

for column, max_length in max_lengths.items():
    if max_length is not None:
        print(f"The maximum length of any element in the {column} column is: {max_length}")

The maximum length of any element in the link_path column is: 2141
The maximum length of any element in the abstract column is: 3209
The maximum length of any element in the all_authors column is: 875
The maximum length of any element in the pdf_path column is: 212
The maximum length of any element in the title column is: 251
The maximum length of any element in the publisher column is: 96
The maximum length of any element in the volume column is: 5
The maximum length of any element in the date_uploaded column is: 10
The maximum length of any element in the keywords column is: 285
The maximum length of any element in the paragraphs column is: 911088
